# Embeddings Quantization

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import uuid
import pandas as pd
from datasets import load_dataset
from fastembed import TextEmbedding
from qdrant_client import models, QdrantClient

from src import load_data, embeddings


# Params

In [16]:
RETRIEVE_K=5
SMPL_SIZE = 25  # int or None

# Clients

In [15]:
client_qdrant = QdrantClient(":memory:") 


# Load and Prepare Data

In [8]:
df_ragquas = load_data.ragquas()
df_docs = load_data.prepare_ragquas(df_ragquas)
df_docs.info()

Raw shape: (250, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     183 non-null    int64 
 1   topic     183 non-null    object
 2   variant   183 non-null    object
 3   question  183 non-null    object
 4   answer    183 non-null    object
 5   j         183 non-null    int64 
 6   context_  183 non-null    object
 7   text_     183 non-null    object
 8   link_     183 non-null    object
 9   uuid      183 non-null    object
dtypes: int64(2), object(8)
memory usage: 14.4+ KB


In [12]:
if SMPL_SIZE:
    df_docs_smpl = df_docs.iloc[:SMPL_SIZE]
else:
    df_docs_smpl = df_docs

# Embeddings

In [13]:
encoder = TextEmbedding(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    )


dn_embeddings = list(encoder.embed(df_docs_smpl['text_']))

print(f"{len(dn_embeddings)=}")
print(f"{len(dn_embeddings[0])=}")

C:\Users\manua\AppData\Local\Temp\ipykernel_14024\315642572.py:1: UserWarning: The model sentence-transformers/paraphrase-multilingual-mpnet-base-v2 now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  encoder = TextEmbedding(


len(dn_embeddings)=25
len(dn_embeddings[0])=768


In [14]:
lst_embeddings = []

for idx in range(len(dn_embeddings)):
    dn_vec = dn_embeddings[idx]

    lst_embeddings.append(
        dn_vec.tolist()
    )

lst_pts = embeddings.create_index_points(
    lst_embeddings,
    df_docs_smpl,
)

lst_qdrant_pts = embeddings.convert_to_qdrant_points(lst_pts)

print(f"{len(lst_qdrant_pts)=}")

len(lst_qdrant_pts)=25


In [ ]:
client_qdrant.create_collection(
    collection_name='raw',
    vectors_config=models.VectorParams(
            size=encoder.embedding_size, 
            distance=models.Distance.COSINE,
        )
)

client_qdrant.create_collection(
    collection_name='q2',
    vectors_config=models.VectorParams(
            size=encoder.embedding_size, 
            distance=models.Distance.COSINE,
        ),
    quantization_config=models.BinaryQuantization(
        binary=models.BinaryQuantizationConfig(
            always_ram=True,
        ),
    ),
)

client_qdrant.upsert(
    collection_name='raw',
    points=lst_qdrant_pts
)

client_qdrant.upsert(
    collection_name='q2',
    points=lst_qdrant_pts
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:


client_qdrant.upsert(
    collection_name='q2',
    points=lst_qdrant_pts
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [50]:
_q_text = df_docs_smpl.sample(1)['question']

In [51]:
_q_vec = next(encoder.embed(_q_text))

print(f"Question: {_q_text}")
embeddings.check_query_qdrant(
        client_qdrant,
        "raw",
        _q_vec,
        retrieve_k=2,
)

Question: 22    necesito buscar un parking o algo en la calle ...
Name: question, dtype: object
point.id='8e281b3c-4543-4a86-a9c9-fdf1d0c0dc26'
point.score=0.35722828374750437
Doc: DESDE 15€/DÍA

Parking en madrid
Encuentra el parking más cercano a tu destino.
PARKING EN LA ZONA DE LA CALLE BAILÉN
Parking en la zona de la calle Bailén
Buscar aparcamiento en Madrid se convierte, diariamente, en un problema para numerosos conductores. Tanto conductores que conocen la zona donde quieren aparcar, como otros que la visitan por primera vez, hallan innumerables problemas sobre todo en días de mucha afluencia de tráfico. Mediante un parking en la zona de la calle Bailén, todo aque...
idx: 20, j: 2
------------------------------
point.id='1812fcc9-5076-4aa1-8bf0-a246229e380e'
point.score=0.24028218377253674
Doc: Parking Plaza de Oriente APK2
Calle de Bailén, 6
186 opiniones
icon contactless
Entrada y salida sin contacto
Puedes entrar y salir de este parking de manera automática, sin necesidad d

In [52]:
_q_vec = next(encoder.embed(_q_text))

print(f"Question: {_q_text}")
embeddings.check_query_qdrant(
        client_qdrant,
        "q2",
        _q_vec,
        retrieve_k=2,
)

Question: 22    necesito buscar un parking o algo en la calle ...
Name: question, dtype: object
point.id='8e281b3c-4543-4a86-a9c9-fdf1d0c0dc26'
point.score=0.35722828374750437
Doc: DESDE 15€/DÍA

Parking en madrid
Encuentra el parking más cercano a tu destino.
PARKING EN LA ZONA DE LA CALLE BAILÉN
Parking en la zona de la calle Bailén
Buscar aparcamiento en Madrid se convierte, diariamente, en un problema para numerosos conductores. Tanto conductores que conocen la zona donde quieren aparcar, como otros que la visitan por primera vez, hallan innumerables problemas sobre todo en días de mucha afluencia de tráfico. Mediante un parking en la zona de la calle Bailén, todo aque...
idx: 20, j: 2
------------------------------
point.id='1812fcc9-5076-4aa1-8bf0-a246229e380e'
point.score=0.24028218377253674
Doc: Parking Plaza de Oriente APK2
Calle de Bailén, 6
186 opiniones
icon contactless
Entrada y salida sin contacto
Puedes entrar y salir de este parking de manera automática, sin necesidad d